In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 25362059
paper_name = 'uehara_shimoi_2015' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_csv('raw_data/HEMF_production_capacity.txt', header=None, names=['orf','data'], sep='\t')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 14 x 2


In [7]:
original_data['orf'] = original_data['orf'].astype(str)

In [8]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [9]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [10]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [orf, data]
Index: []


In [11]:
original_data.set_index('orf', inplace=True)

In [12]:
# These data represet a production capacity ratio (Hmut/Hwt). Shift data by 1 so that the WT phenotype is 0 (by convention)
original_data = original_data-1

# Prepare the final dataset

In [15]:
data = original_data.copy()

In [16]:
dataset_ids = [16495]
datasets = datasets.reindex(index=dataset_ids)

In [17]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [18]:
data.head()

dataset_id,16495
data_type,value
orf,
YOL086C,9.7
YGR204W,5.3
YAR015W,3.7
YGR061C,3.5
YMR300C,0.9


## Subset to the genes currently in SGD

In [19]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [20]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16495
,data_type,value
gene_id,orf,
5135,YOL086C,9.7
3149,YGR204W,5.3
67,YAR015W,3.7
3006,YGR061C,3.5
4604,YMR300C,0.9


# Normalize

In [23]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [24]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [25]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      16495           
data_type       value     valuez
gene_id orf                     
5135    YOL086C   9.7  49.416118
3149    YGR204W   5.3  27.000559
67      YAR015W   3.7  18.849447
3006    YGR061C   3.5  17.830558
4604    YMR300C   0.9   4.585001

# Print out

In [26]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [27]:
from IO.save_data_to_db3 import *

In [28]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 25362059...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00, 11.65it/s]

Updating the data_modified_on field...
